*2021/01/09 Sat*

# 4-4. 스타크래프트를 만들자 2 (const, static)

## 생성자의 초기화 리스트(initializer list)

In [ ]:
Marine::Marine(int coord_x, int coord_y)
    : coord_x(coord_x), coord_y(coord_y), is_dead(false) {}

// 아래 코드와 동일.

Marine::Marine(int _coord_x, int _coord_y) {
    coord_x = _coord_x;
    coord_y = _coord_y;
    is_dead = false;
}

/*
위는 int a = 10;, 아래는 int a; a = 10;의 느낌.
레퍼런스나 상수는 생성과 동시에 초기화되어야 했음. 복사 생성자도 그렇고. 이럴 때 위처럼 초기화 리스트를 쓰는 것.
*/

In [ ]:
class Test {
    int i;
    public:
        Test(int i) : i(i) {}
};
int main() {
    Test a = 10; // 가능 -> Test a(10);
    // int i(10);도 됨.
}

## static 변수

클래스 하나에만 종속되는 변수. 클래스의 모든 객체들이 공유하는 변수.

In [ ]:
class Marine {
    static int total_marine_num;
    // static 변수를 클래스 내부에서 초기화하는 건 불가능. const static이면 가능 (e.g., const static int x = 0;)
    
    public:
        Marine();
        Marine(int x, int y);
        Marine(int x, int y, int default_damage);
    
        ~Marine() { total_marine_num--; }
};

////
int Marine::total_marine_num = 0;
////

Marine::Marine()
    : hp(50), coord_x(0), coord_y(0), default_damage(5), is_dead(false) {
    total_marine_num++;
}
...

* static 함수

```static T func(arg, ...);```

호출 : ```(객체).(멤버 변수/함수)``` vs ```(클래스)::(static 변수/함수)```

## this 키워드

In [3]:
class Marine {
    ...
    public:
        Marine& be_attacked(int damage_earn);
    ...
};

Marine& Marine::be_attacked(int damage_earn) {
    hp -= damage_earn; // this->hp -= damage_earn;
    if (hp <= 0) is_dead = true; // if (this->hp <= 0) this->is_dead = true;
            
    return *this;
}

* 레퍼런스를 리턴하는 함수

In [ ]:
class Test {
    int x;
    ...
    public:
        int& access_x() { return x; }
        int get_x() { return x; }
        void show_x() { std::cout << x << std::endl; }
    ...
};

Test a;
int& d = a.get_x(); // 컴파일 오류 : int&는 lvalue에 대한 레퍼런스, a.get_x()는 rvalue
d = 4;

부연 설명 - 좌측값은 어떠한 메모리를 가리키는데, & 연산자를 통해 그 위치를 참조할 수 있다. 우측값은 좌측값이 아닌 값들이다.

## 상수 함수

변수들의 값을 바꾸지 않고 읽기만 하는, 마치 상수 같은 함수.

In [ ]:
class Test {
    public:
        int attack() const;
};

int Test::attack() const { return default_damage; }

## 생각해보기

아래와 같은 코드에서 *복사 생성*은 몇 번이나 표시될까요?

In [ ]:
#include <iostream>

class A {
  int x;

 public:
  A(int c) : x(c) {}
  A(const A& a) {
    x = a.x;
    std::cout << "복사 생성" << std::endl;
  }
};

class B {
  A a;

 public:
  B(int c) : a(c) {}
  B(const B& b) : a(b.a) {}
  A get_A() {
    A temp(a);
    return temp;
  }
};

int main() {
  B b(10);

  std::cout << "---------" << std::endl;
  A a1 = b.get_A();
}

-> 3번(A temp(a); return temp; A a1 = b.get_A();) (return temp; -> temp -> 임시 객체)

단, 컴파일러는 copy elision이라는 기술 사용해서 불필요한 복사 막고 있음. -> temp가 a1 객체의 복사 생성자에 바로 들어간다 생각하면 2번이 나올 수도 있는 것.

# 5. 내가 만드는 String 클래스

In [ ]:
while (some_condition) {
  str.insert(some_location, 'a');
}

'insert 작업에서의 잦은 할당/해제를 피하기 위해 미리 메모리를 할당해놓기' 와 '메모리를 할당해 놓되, 많은 자원을 낭비하지 않는다' 라는 두 조건을 모두 만족하는 방법이 있을까요? 물론 있습니다. 메모리를 미리 할당할 경우, 현재 메모리 크기의 두 배 정도를 할당해 놓는다는 것입니다.

In [ ]:
if (string_length + str.string_length > memory_capacity) {
    // 이제 새롭게 동적으로 할당을 해야 한다.

    if (memory_capacity * 2 > string_length + str.string_length)
      memory_capacity *= 2;
    else
      memory_capacity = string_length + str.string_length;
...

위와 같은 방법은 C++에서 동적으로 할당되는 메모리를 처리하는 데 매우 빈번하게 사용되는 기법 중 하나!

문자열을 검색하는 알고리즘은 수 없이 많지만, 어떤 상황에 대해서도 좋은 성능을 발휘하는 알고리즘은 없습니다. (예를 들어 짧은 문자열 검색에 최적화 된 알고리즘과 긴 문자열 검색에 최적화 된 알고리즘들 같이 말입니다) 그렇기에 특별한 알고리즘을 사용하는 경우에는 그 클래스의 사용 목적이 명확해서 그 알고리즘이 좋은 성능을 발휘할 수 있는 경우에만 사용하는 것이 보통입니다. 따라서 우리의 MyString 의 경우, 가장 간단한 방법으로 find 알고리즘을 구현하기로 하였습니다.

여태까지 배운 C++ 에 대한 내용을 종합해서 훌륭한 MyString 클래스를 만들었다고 볼 수 있습니다. 우리의 MyString 클래스는 다음과 같은 인터페이스를 제공합니다.
* 문자 c 혹은 C 형식 문자열 str 에서 생성할 수 있는 생성자와 복사 생성자
* 문자열의 길이를 리턴하는 함수(length)
* 문자열 대입 함수(assign)
* 문자열 메모리 할당 함수(reserve) 및 현재 할당된 크기를 알아오는 함수(capacity)
* 특정 위치의 문자를 리턴하는 함수(at)
* 특정 위치에 특정 문자열을 삽입하는 함수(insert)
* 특정 위치의 특정 개수의 문자를 지우는 함수(erase)
* 특정 위치를 시작으로 특정 문자열을 검색하는 함수(find)
* 두 문자열을 사전식 비교하는 함수(compare)

## 생각해보기

여러가지 검색 알고리즘(KMP, Boyer - Moore) 들을 이용하는 find 함수를 만들어보세요. 어떤 알고리즘의 경우 미리 계산된 테이블이 필요할 텐데, 이러한 정보들 역시 class 변수로 처리하셔도 됩니다. (난이도 : 上)

# 4-6. 클래스의 explicit과 mutable 키워드

## explicit

In [ ]:
class MyString {
    ...
public:
    MyString(int capacity);
    void DoSomethingWithString(MyString s);
    ...
}

int main()
{
    ...
    DoSomethingWithString(3); // 실수 -> 그러나 컴파일러는 3을 MyString(3)으로 implicit conversion.
    ...
}

C++ 컴파일러는 꽤 똑똑해서 위와 같이 오류가 나야 하는 코드도 적당한 생성자를 찾고 암시적 변환해서 컴파일.

-> ```explicit``` 키워드를 통해 원치 않는 암시적 변환을 할 수 없도록 컴파일러에게 명시할 수 있음.

In [ ]:
class MyString {
    ...
public:
    explicit MyString(int capacity);
    void DoSomethingWithString(MyString s);
    ...
}

int main()
{
    ...
    DoSomethingWithString(3); // error : no matching function for call to 'DoSomethingWithString'
    ...
}

## mutable

외부에서 보기에 상수 함수일지라도, 내부에서 데이터베이스에 대한 cache 변수가 존재해서 그것을 업데이트하는 작업과 같은 것을 수행하는 애매한 경우.

In [ ]:
class Server {
    // ... (생략) ...
    
    mutable Cache cache; // mutable 키워드 선언
    
    User GetUserInfo(const int user_id) const {
        Data user_data = cache.find(user_id);
        
        if (!user_data) {
            user_data = Database.find(user_id);
            
            cache.update(user_id, user_data); // mutable이 아니었다면, 내부 상태를 변경하는 동작이므로 불가능
        }
        
        return User(user_data);
    }
};
